In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from pypokerengine.players import BasePokerPlayer
from pypokerengine.utils.card_utils import Card, Deck
from pypokerengine.api.game import setup_config, start_poker

import pickle
import tensorflow as tf
import random

import sys
sys.path.insert(0, '../scripts/')

import PlayerModels as pm
from MyEmulator import MyEmulator
from DQNPlayer import DQNPlayer
from util import *

## Initialization

In [2]:
h_size = 128

In [13]:
%time main_wp = DQNPlayer(h_size=h_size, is_restore=True, is_train=False, debug=True)

INFO:tensorflow:Restoring parameters from ../cache/models/model_100000.ckpt
CPU times: user 764 ms, sys: 64 ms, total: 828 ms
Wall time: 804 ms


## Testing

In [14]:
config = setup_config(max_round=50, initial_stack=1500, small_blind_amount=15, summary_file='/dev/null')

config.register_player(name="wp", algorithm=main_wp)
# config.register_player(name="r2", algorithm=RandomPlayer())
config.register_player(name="f2", algorithm=pm.CallPlayer())
config.register_player(name="f3", algorithm=pm.CallPlayer())
config.register_player(name="f4", algorithm=pm.CallPlayer())
config.register_player(name="f5", algorithm=pm.CallPlayer())
config.register_player(name="f6", algorithm=pm.CallPlayer())
config.register_player(name="f7", algorithm=pm.CallPlayer())
config.register_player(name="f8", algorithm=pm.CallPlayer())
config.register_player(name="f9", algorithm=pm.CallPlayer())

game_result = start_poker(config, verbose=1)

(['Hole:', ['H5', 'D8']],)
(['Start stack:', 1500],)
(['Estimation:', 0.0817],)
Started the round 1
Street "preflop" started. (community card = [])
"f4" declared "call:30"
"f5" declared "call:30"
"f6" declared "call:30"
"f7" declared "call:30"
"f8" declared "call:30"
"f9" declared "call:30"
(array([ -12.28872776,   -0.70328993,  -53.35214615, -444.25775146,
        -97.49479675], dtype=float32),)
"wp" declared "call:30"
"f2" declared "call:30"
"f3" declared "call:30"
Street "flop" started. (community card = ['DQ', 'C7', 'S4'])
"f2" declared "call:0"
"f3" declared "call:0"
"f4" declared "call:0"
"f5" declared "call:0"
"f6" declared "call:0"
"f7" declared "call:0"
"f8" declared "call:0"
"f9" declared "call:0"
(array([ -29.64264297,   14.76653576,  -44.00888062, -582.73480225,
       -129.59130859], dtype=float32),)
"wp" declared "call:0"
Street "turn" started. (community card = ['DQ', 'C7', 'S4', 'D3'])
"f2" declared "call:0"
"f3" declared "call:0"
"f4" declared "call:0"
"f5" declared "c

## Metric

In [16]:
%time main_wp = DQNPlayer(h_size=h_size, is_restore=True, is_train=False, debug=False)

INFO:tensorflow:Restoring parameters from ../cache/models/model_100000.ckpt
CPU times: user 736 ms, sys: 48 ms, total: 784 ms
Wall time: 770 ms


In [18]:
config = setup_config(max_round=50, initial_stack=1500, small_blind_amount=15, summary_file='/dev/null')

config.register_player(name="wp", algorithm=main_wp)
# config.register_player(name="r2", algorithm=RandomPlayer())
config.register_player(name="f2", algorithm=pm.CallPlayer())
config.register_player(name="f3", algorithm=pm.CallPlayer())
config.register_player(name="f4", algorithm=pm.CallPlayer())
config.register_player(name="f5", algorithm=pm.CallPlayer())
config.register_player(name="f6", algorithm=pm.CallPlayer())
config.register_player(name="f7", algorithm=pm.CallPlayer())
config.register_player(name="f8", algorithm=pm.CallPlayer())
config.register_player(name="f9", algorithm=pm.CallPlayer())

In [37]:
stacks = []
for i in range(100):
    game_result = start_poker(config, verbose=0)
    stacks.append([res for res in game_result['players'] if res['name'] == 'wp'][0]['stack'])

In [38]:
np.mean(stacks), np.std(stacks)

(2160.0, 4949.1817505523068)